In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Luke_violence_lang") #changing working directory to google drive

In [ ]:
import pandas as pd
import os # for chnaging working directory we need to import os
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def remove_content(text):
    text = re.sub(r"http\S+", "", text) #remove urls.\S+ matches any character that isn't whitespace & is opposite of\s
    text=re.sub(r'\S+\.com\S+','',text) #remove urls."" mean no space or nothing
    text=re.sub(r'\@\w+','',text) #remove mentions
    text =re.sub(r'\#\w+','',text) #remove hashtags
    text =re.sub(r"^RT\s+",'',text) #remove rt's
    text = re.sub('[^A-Za-z]', ' ', text.lower())#remove non-alphabets(?), Lower is method applied to string text; matching each word in text-replacing with lower version
    return text

In [ ]:
df_violent_phrases=pd.read_excel("lov_terms_groups.xlsx")

In [ ]:
df_violent_phrases["Terms_1"] = df_violent_phrases["Terms_1"].astype(str)
df_violent_phrases["Terms_2"] = df_violent_phrases["Terms_2"].astype(str)

In [ ]:
df_violent_phrases["Terms_1"]=df_violent_phrases["Terms_1"].replace("nan","")
df_violent_phrases["Terms_2"]=df_violent_phrases["Terms_2"].replace("nan","")

In [ ]:
df_violent_phrases.head(5)

,Unnamed: 0,Category,Subcategory,Terms_1,Terms_2
0,1,Group_Alignment,In_Group_Identification,We,European
1,2,Group_Alignment,In_Group_Identification,We,cultural conservative
2,3,Group_Alignment,In_Group_Identification,We,Christian conservative
3,4,Group_Alignment,In_Group_Identification,We,conservative
4,5,Group_Alignment,In_Group_Identification,We,indigenous


In [ ]:
df_violent_phrases["cleaned_t1"]=df_violent_phrases["Terms_1"].apply(lambda x: remove_content(x))
df_violent_phrases["cleaned_t2"]=df_violent_phrases["Terms_2"].apply(lambda x: remove_content(x))

In [ ]:
df_violent_phrases["combined_phrases"] = df_violent_phrases["cleaned_t1"] + " " + df_violent_phrases["cleaned_t2"]

In [ ]:
df_violent_phrases = df_violent_phrases.rename(columns={'Unnamed: 0': 'Khrys_Index'})

In [ ]:
df_messages=pd.read_excel("dummy_data.xlsx")

In [ ]:
df_messages

,Message,identifier
0,we Europeans don’t want immigrants,8
1,the best thing is cultural Conservative,2
2,We non-muslims are best,555
3,We people for the people,4
4,We peoples for the people,101
5,We anon protect the world,6
6,We protect anon from world,7
7,We east european,20
8,gas Chambering is okay,9
9,gas chambers,678


In [ ]:
df_messages = df_messages.sample(frac=1)  # 20% of the rows

In [ ]:
df_messages["Message"] = df_messages["Message"].astype(str)

In [ ]:
df_messages["cleaned_messages"]=df_messages["Message"].apply(lambda x: remove_content(x))

Modified to match phrase even with one word in between and one word after the phrase (Final Selected)

In [ ]:
def match_violent_phrases_oneword(term1,term2,message):
  """
  Matches phrases in a text with just one word between the words of phrases (between t1 and t2).

  Args:
    text: The messages to search for phrases in.
    phrases: violent phrase to match.

  Returns:
    matched phrase.
  """
  #match = re.search(r"\b{}\s+(?:\w+\s+)?{}\b".format(term1, term2),message)
  #match = re.search(r"\b{}(s|ing|ed|er|es)?\s+(?:\w+\s+)?{}(s|ing|ed|er|es)?\b".format(term1, term2), message)
  if term2 == "":
    match = re.search(r"\b{}(s|ing|ed|er|es)?\b".format(term1), message)
  else:
    match = re.search(r"\b{}(s|ing|ed|er|es)?\s+(?:\w+\s+)?{}(s|ing|ed|er|es)?\b".format(term1, term2), message)

  if match:
    matched_phrase_word = match.group()
    return matched_phrase_word
  else:
    return None  # Return None if no match is found

In [ ]:
# Example usage
message = "a text book"
message2 = "a text"
message3 = "a big text"
message4 = "a big text book with pictures"
message5 = "This is a sample text with some violence-related phrases."
t1 = "a"
t2 = "text"
matched_phrases = match_violent_phrases_oneword(t1,t2, message)
print("Matched phrases:", matched_phrases)
matched_phrases2 = match_violent_phrases_oneword(t1,t2, message2)
print("Matched phrases:", matched_phrases2)
matched_phrases3 = match_violent_phrases_oneword(t1,t2, message3)
print("Matched phrases:", matched_phrases3)
matched_phrases4 = match_violent_phrases_oneword(t1,t2, message4)
print("Matched phrases:", matched_phrases4)
matched_phrases5 = match_violent_phrases_oneword(t1,t2, message5)
print("Matched phrases:", matched_phrases5)

Matched phrases: a text
Matched phrases: a text
Matched phrases: a big text
Matched phrases: a big text
Matched phrases: a sample text


In [ ]:

messages =[]
violent_phrases_term1=[]
violent_phrases_term2=[]
matching = []
phrase_counter =[]
identifier_col = [] # to store the identifier from messages
t1_list=[]
t2_list=[]
phrase_ident = []

for i in range(len(df_violent_phrases)):
  t1 = str(df_violent_phrases["cleaned_t1"].iloc[i])
  t2 = str(df_violent_phrases["cleaned_t2"].iloc[i])
  Khrys_iden = df_violent_phrases["Khrys_Index"].iloc[i]
  k=0
  for j in range(len(df_messages)):
    message = df_messages["cleaned_messages"].iloc[j]
    original_index = df_messages["identifier"].iloc[j]
    matches = match_violent_phrases_oneword(t1,t2,message)
    if matches:

      k=k+1
      messages.append(message)
      identifier_col.append(original_index) # Store the identifier
      matching.append(matches)
      t1_list.append(t1)
      t2_list.append(t2)
      phrase_ident.append(Khrys_iden)

  phrase_counter.append(k) # This is tracking number of time a phrase has occured in messages
  #print(k)
  violent_phrases_term1.append(t1)
  violent_phrases_term2.append(t2)


In [ ]:
df_violent_phrase_counter= pd.DataFrame(list(zip(violent_phrases_term1,violent_phrases_term2, phrase_counter)),
               columns =['Violent Phrases Term1', 'Violent Phrases Term2','counter'])

In [ ]:
df_message_match= pd.DataFrame(list(zip(messages, matching, identifier_col, t1_list, t2_list, phrase_ident)),
               columns =['Message', 'Matched Phrase', "Original Index", "Term 1", "Term 2", "Khrys_identifier"])

In [ ]:
df_message_match.to_excel("Messages_Matched_Phrases.xlsx")

In [ ]:
df_violent_phrase_counter.to_excel("Violent_Phrases_Counter.xlsx")